# Query Service
Query automatic synaptic partner predictions and integrate with CATMAID

## Queries

  1. get all connections between a given pair of skeletons,
  2. get all upstream connections for a given skeleton
  3. get all downstream connections for a given skeleton
  4. get all upstream/downstream connections for a given skeleton

## Imports

In [7]:
# pip install git+git://github.com/schlegelp/pymaid@master
import pymaid

## Procedure

* Get skeleton from CATMAID
* Look up segmentation IDs for all skeleton nodes from FAFB-FFN segmentation using CloudVolumeServer / fafbseg-py
* Retrieve all synaptic links for the set of segmentationIDs (from a local sqlite database with the prepopulated lookups from synaptic link to segmentation ID)
* Associate the synaptic links with the the corresponding skeleton
* Discard synaptic links that stem from split-away parts of segments with false mergers
* Push the data to CATMAID using PyMAID